In [3]:
import matplotlib.pyplot as plt 
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from numpy import sin, pi, cos, log

In [4]:
class Data(Dataset):
    ## constructor
    def __init__(self, points = 128):
        self.x = torch.zeros ((points, 1))
        self.y = torch.zeros ((points, 1))
        a = np.arange (0, 2*pi, pi/(points/2))
        for i in range (points):
            self.x[i] = a[i]
            self.y[i] = sin(self.x[i])**2 
        # self.step = pi/(points/2)
        # self.X = np.arange(self.step, 2*pi+ self.step, self.step)
        # self.Y = np.square (sin(self.X))
        # self.y = torch.from_numpy(self.Y)
        # self.x = torch.from_numpy(self.X)
        self.len = self.y.shape[0]

    ## getter
    ## todo, 
    def __getitem__(self, index):
        return self.x[index], self.y[index]

    ## get length
    def __len__ (self):
        return self.len

    ##plot the diagram
    def plot_diagram (self):
        plt.plot(self.x, self.y)
        plt.xlabel("$//theta$")
        plt.ylabel("$f$")
        plt.show()

In [5]:
class Net(nn.Module):
    
    ## define the model and the dimension of parameters for each layers
    def __init__(self, Layers):
        super(Net, self).__init__()
        self.hidden = nn.ModuleList()
        for input_size, output_size in zip(Layers, Layers[1:]):
            self.hidden.append(nn.Linear(input_size, output_size))


    def forward (self, activation): 
        L = len (self.hidden)
        for (l, linear_transform) in zip(range (L), self.hidden):
            if l < L - 1:
                activation = F.relu(linear_transform(activation))
            else:
                activation = F.sigmoid(linear_transform(activation))
        return activation

In [9]:
def train(data_set, model, criterion, train_loader, optimizer, epochs = 100):

    Loss = []
    for epochs in range (epochs):
        for x, y in train_loader:
            yhat = model(x)
            loss = criterion(yhat, y)
            optimizer.zero_grad ()
            loss.backward()
            optimizer.step()
            Loss.append(loss.item())

        if epochs % 10 == 0:
            plt.cla()
            plt.scatter(x, y, marker=".")
            plt.plot(x, model(x), "r-", lw= 5)
            plt.text(0.5, 0, 'Loss=%.4f' % loss.data.numpy(), fontdict={'size': 20,             'color':  'red'})
            plt.pause(0.1)
    plt.ioff()
    plt.show()



In [10]:
Layers = [1, 16, 1]
model = Net(Layers)
learning_rate = 0.10
data_set = Data()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
train_loader = DataLoader(dataset=data_set)
criterion = nn.MSELoss()
Loss = train(data_set, model, criterion, train_loader, optimizer, epochs=1000)




RuntimeError: Can't call numpy() on Tensor that requires grad. Use tensor.detach().numpy() instead.